In [130]:
import numpy as np
import rlwe
import lwe

In [131]:
def polynomial_to_string(p):
    terms = []
    for i, coeff in enumerate(p.coeff):
        if coeff != 0:  # We only add non-zero terms
            if i == 0:
                terms.append(f"{coeff}")
            elif i == 1:
                terms.append(f"{coeff}x")
            else:
                terms.append(f"{coeff}x^{i}")
    # Join all terms, handling the signs correctly
    return " + ".join(terms).replace("+-", "- ")

In [132]:
lwe_config = lwe.LweConfig(dimension=1024, noise_std=2 ** (-24))

In [133]:
lwe_key = lwe.generate_lwe_key(lwe_config)

In [134]:
#convert lwe key to rlwe key
rlwe_key = rlwe.convert_lwe_key_to_rlwe(lwe_key)

In [135]:
gsw_config = rlwe.GswConfig(rlwe_config=rlwe_key.config, log_p=8)

In [136]:
gsw_key = rlwe.convert_rlwe_key_to_gsw(rlwe_key, gsw_config)

In [137]:
#choosing polynomal messages f and g as 2 and 3x
f = rlwe.build_monomial(c=1, i=0, N=rlwe_key.config.degree)
g = rlwe.build_monomial(c=3, i=1, N=rlwe_key.config.degree)
print(polynomial_to_string(f), "\n", polynomial_to_string(g))

1 
 3x


In [138]:
gsw_plaintext = rlwe.GswPlaintext(config=gsw_config, message=f)
rlwe_plaintext = rlwe.rlwe_encode(g, rlwe_key.config)

In [139]:
gsw_ciphertext = rlwe.gsw_encrypt(gsw_plaintext, gsw_key)
rlwe_ciphertext = rlwe.rlwe_encrypt(rlwe_plaintext, rlwe_key)

In [140]:
rlwe_ciphertext_prod = rlwe.gsw_multiply(gsw_ciphertext, rlwe_ciphertext)

fg = rlwe.build_monomial(c=2, i=1, N=rlwe_key.config.degree)

In [141]:
p = rlwe.rlwe_decode(rlwe.rlwe_decrypt(rlwe_ciphertext_prod, rlwe_key))
print(p)

Polynomial(N=1024, coeff=array([0, 3, 0, ..., 0, 0, 0]))


In [142]:
polynomial_to_string(p)

'3x'